# AiDM 2019 Group 26: Assignment 2: LSH for the Netflix data

## Part 2: Local Sensitive Hashing.

Auke Bruinsma, s1594443 and Simon van Wageningen, s2317079.

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import time,sys
import numpy.ma as ma
from scipy.sparse import csc_matrix
from collections import defaultdict
import itertools as it

We have divided this part of the assignments in 5 parts:

1. Importing the data
2. Signature Matrix
3. LSH
4. Jaccard Similarity
5. Run the algorithm

For each part we will be explaining what we are doing.

### 1. Importing the data

This part is pretty straight forward. First we import the data. Since this is already in 'npy' format, it's easy to work with. We define some global variables so the code is more clear and print some data so we can have a look at how the data looks.

In [2]:
# Import data.
dataloc = 'data/user_movie.npy' # Location of the data file in string format.
data = np.load(dataloc) # Import the data using numpy.load.

In [3]:
# Some global variables.
n_users = np.max(data[:,0])+1
n_movies = np.max(data[:,1])+1

In [4]:
# Print some data just for a quick overview.
print('Number of users: {0}'.format(n_users))
print('Number of movies: {0}'.format(n_movies))
print('\nData:\n\n{0}'.format(data))

Number of users: 103703
Number of movies: 17770

Data:

[[     0     29]
 [     0    156]
 [     0    172]
 ...
 [103702  17621]
 [103702  17626]
 [103702  17763]]


### 2. Signature Matrix.

Next, we need to compute the signature matrix. Since we need to find pairs within 15 minutes it is important that this part does not take much time per iteration. We have rewritten this function a lot of times because we needed it faster. The function now takes approximately 1 second for 1 iteration. With iteration we mean 1 permutation of the random array used for minhashing. It works as follows:

1. First we initialise the signature matrix and permutation array with the correct sizes. We use 100 different permuations/iterations for the randomly shuffled array. This seems as a nice approximation of the jaccard similarity to us. It also a good number to work with, since it can be divided by 4 and 5 (which is nice because we need to create bands/buckets later).

2. We permute the array X so that the index for the first 1 for a user is what we need to obtain.

3. We find this first '1', using the argmax function. Since the array X only contains ones, it will automatically return the first one it finds, which is the '1' we need. We do not apply the argmax function to all the 17000 movies a user has watched. This would take way too long. Instead, we apply it to the first 500 movies a user has watched. This is the check_length variable. Since a user has watched between 300 and 3000 movies, most of the times the argmax function will find the first '1' in the first 500 movies. For the users it doesn't find a '1', a for loop is initiated which stars at movie 500 and iterates until it finds a '1'. For-loops are slow, but since almost all of the '1's are found by the argmax function, the for loop relatively takes no time. We tried for different values for the check_length parameter. The value you choose sets the ratio of '1's found by the argmax function and '1' found by the for-loop. Since almost all the '1's are found by the argmax function, you do not want to set the check_length parameter to a high value, since then the algorithm would take a lot of time. You also do not want to set it too low, then a lot of '1's are found by the for-loop. Somewhere there is a balance between the two methods. We found that 500 was a good value.

4. Eventually we put all the signatures in the signature array and return it.

In [5]:
def signatures(X,iter):
    sig_mat = np.zeros((n_users,iter))    
    permutation = np.arange(0,n_movies)
    check_length = 500

    def min_hash(X,sig_mat,i):
        sig = X[:,:check_length].argmax(axis=1)
        mask = ((sig == 0) & (X[:,0] != 1))
        X_masked = X[mask]
        change = []
        for m in range(len(X_masked)):
            for n in range(check_length-1,len(X_masked[m])):
                if X_masked[m][n] == 1:
                    change.append(n);
                    break;

        sig[mask] = change
        sig_mat[:,i] = sig
        
    for i in range(iter):
        np.random.shuffle(permutation)
        X_perm = X[:,permutation]
        min_hash(X_perm,sig_mat,i)
    
    return sig_mat

### 3. LSH.

The next part is to make bands out of the signature matrix. We take signature matrix with size 100, and divide in 20 bands with size 5. We take the tuple of 5 signatures and list all the users which have these 5 signatures. These are stored in the buckets and the buckets are stored in the hash_table. So the hash_table contains 20 buckets and each bucket contains similar users for each string of 5 signatures. If a piece of 5 signatures only contains 1 user, that user has no one to compare to, so it is deleted.

The hashtable will then be used to make pairs. Each array of users will be combined so all users are paired in the possible ways. This is done using itertools.combinations. Eventually, two arrays will be returned.

In [6]:
def band_partition(sig_mat,bands):
    split_mat = np.split(sig_mat,bands,axis=1)
    hash_table = []
    
    for b in range(bands):
        bucket = defaultdict(list)
        
        for s in range(len(split_mat[b])):
            series = tuple(split_mat[b][s])
            bucket[series].append(s)
                
        for v in list(bucket.keys()):
            if len(bucket[v]) == 1:
                del bucket[v]
    
        hash_table.append(bucket)       

    return hash_table

In [7]:
def make_pairs(hash_table):
    pair_1 = []
    pair_2 = []
    for i in range(len(hash_table)):
        for j in list(hash_table[0].values()):
            combinations = list(it.combinations(j,2))
            for k in range(len(combinations)):
                pair_1.append(combinations[k][0])
                pair_2.append(combinations[k][1])
    
    return pair_1,pair_2

### 4. Jaccard similarity.

The final thing that needs to be done now, is to compute the jaccard similarity for all the pairs. If the jaccard similiary is larger than 0.5 it will be printed and written to a file.

In [8]:
def jsim(X,p1,p2):
    file = open('results.txt','w')
    for i in range(len(p1)):
        intersect = np.sum(X[p1[i]] & X[p2[i]])
        union = np.sum(X[p1[i]] | X[p2[i]])
        if intersect/union > 0.5:
            print('{0} {1} {2} {3}'.format(intersect/union,p1[i],p2[i],i))
            file.write('{0} {1} {2}\n'.format(intersect/union,p1[i],p2[i]))
    file.close()

### 5. Run the Algorithm.

Now we run the algorithm with differing amounts of permutation and bands for experimentation. For each part the time is also outputted so we can see which part of the algorithm takes up most time. The signature matrix takes up most of the time (if we dont count calculating jaccard similarities), which is expected. The other parts only take up seconds. This is nice, because approximately 13 minutes are reserved for finding pairs with jac_sim > 0.5. 

In [9]:
n_iter = 100
bands = 20

In [10]:
%%time
# Create a sparse scipy matrix with 0's and 1's.
X = csc_matrix((np.ones(len(data)),(data[:,0],data[:,1])),dtype=np.int8).toarray()

CPU times: user 5.9 s, sys: 1.89 s, total: 7.79 s
Wall time: 7.79 s


In [11]:
%%time 
# This cell takes approximately 1s per iteration, so 1m40 for 100 permutations.
sig_mat = signatures(X,n_iter)

CPU times: user 2min 22s, sys: 1min 46s, total: 4min 9s
Wall time: 4min 9s


In [12]:
%%time
hash_table = band_partition(sig_mat,bands)

CPU times: user 16 s, sys: 126 ms, total: 16.1 s
Wall time: 16.1 s


In [13]:
%%time
pair_1,pair_2 = make_pairs(hash_table)

CPU times: user 11.6 s, sys: 302 ms, total: 11.9 s
Wall time: 11.9 s


In [14]:
# Some final statistics.
print('Number of users:       {0}'.format(n_users))
print('Number of movies:      {0}'.format(n_movies))
print('Number of iterations:  {0}'.format(n_iter))
print('Number of bands:       {0}'.format(bands))
print('Number of pairs:       {0}'.format(len(pair_1)))
print('Size reduction factor: {0}'.format((((n_users**2-n_users)/2)/len(pair_1))))

Number of users:       103703
Number of movies:      17770
Number of iterations:  100
Number of bands:       20
Number of pairs:       19711360
Size reduction factor: 272.79214894355334


In [ ]:
%%time
jsim(X,np.asarray(pair_1),np.asarray(pair_2))

0.5893909626719057 13620 49650 16576
0.8378839590443686 52494 99903 230596
0.5271493212669683 47112 98667 275175
0.5067079463364293 47929 64255 367034
0.529505582137161 47929 68130 367036
0.5004055150040552 34132 81962 484386
0.5112087912087913 36382 81962 484447
0.5217561740493923 81962 94427 484852
0.5120178870877585 8295 33755 747954
0.5043442283822921 13028 100078 861067
0.5042627091885065 15038 20443 861217
0.5045439246045103 15038 100078 861257
0.578795528308691 7872 18866 886254
0.5893909626719057 13620 49650 1002144
0.8378839590443686 52494 99903 1216164
0.5271493212669683 47112 98667 1260743
0.5067079463364293 47929 64255 1352602
0.529505582137161 47929 68130 1352604
0.5004055150040552 34132 81962 1469954
0.5112087912087913 36382 81962 1470015
0.5217561740493923 81962 94427 1470420
0.5120178870877585 8295 33755 1733522
0.5043442283822921 13028 100078 1846635
0.5042627091885065 15038 20443 1846785
0.5045439246045103 15038 100078 1846825
0.578795528308691 7872 18866 1871822


### Dicussion/Conclusion

- We are finding pairs with jaccard similarity larger than 0.5 so we conclude that we have correctly implemented the algorithm. 
- This algorithm is timewise a large improvement compared to the brute force method.
- We do not know how much if the users have JS > 0.5, so it is hard to know if we find all pairs with JS > 0.5 or are maybe missing some. However there is not really a way to check this, since what we have implemented is the fastet way we currently have of finding these pairs. Since we find a couple pairs with JS > 0.5 each minute, we believe everything works as intended.